# `aiohttp` -- Async HTTP client/server for asyncio and Python

In [1]:
import aiohttp

## HTTP client example
The examples use http://httpbin.org/, a simple service for testing requests.

### GET request with parameters, use response in text form

In [2]:
async with aiohttp.ClientSession() as session:
    async with session.get("http://httpbin.org/get", params={"param1": "value1"}) as response:
        print(f"Response status: {response.status}")
        print(await response.text())

Response status: 200
{
  "args": {
    "param1": "value1"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "Python/3.7 aiohttp/3.5.4"
  }, 
  "origin": "80.87.168.98, 80.87.168.98", 
  "url": "https://httpbin.org/get?param1=value1"
}



### GET request, parse JSON response as Python object

In [3]:
async with aiohttp.ClientSession() as session:
    async with session.get("http://httpbin.org/get", params={"param1": "value1"}) as response:
        data = await response.json()

print(data["headers"]["User-Agent"])

Python/3.7 aiohttp/3.5.4


### POST request

In [4]:
async with aiohttp.ClientSession() as session:
    async with session.post("http://httpbin.org/post", data=b"Input data for the HTTP POST request") as response:
        data = await response.json()

print(f"Content of the 'data' field in the response: '{data['data']}'")

Content of the 'data' field in the response: 'Input data for the HTTP POST request'


## HTTP server example

In [5]:
from aiohttp import web
app = web.Application()

### Define a simple request hander that ignores input and returns text

In [6]:
async def hello(request):
    return web.Response(text="Hello")

### Add the handler to the routes of the server application

In [7]:
app.add_routes([web.get("/hello", hello)])

### GET request with parameters in the URL path and optional query parameters

In [8]:
async def hello2(request):
    first_name = request.match_info["first"]
    last_name = request.match_info["last"]
    query = request.rel_url.query
    
    params = {key: value for key, value in query.items()}

    return web.json_response(
        {
            "message": f"Hello {first_name} {last_name}!",
            "parameters": params
        })

app.add_routes([web.get("/hello/{first}/{last}", hello2)])

### Start the HTTP server
Note that this is much easier usually:

    web.run_app(app, port=port)
    
starts the server on the given port and starts an event loop for handling requests. Here we have to do it a bit differently because Jupyter is already running an event loop:

In [9]:
runner = aiohttp.web.AppRunner(app)
await runner.setup()
site = aiohttp.web.TCPSite(runner, "localhost", 8090)    
await site.start()

Now the server is reachable via http://localhost:8090/hello (provided the port was not taken yet and is reachable from the outside). You can try it in the browser!

We can also access it with HTTP requests directly from this notebook:

In [10]:
async with aiohttp.ClientSession() as session:
    async with session.get("http://localhost:8090/hello/John/Smith", params={"extra-param": "param-value"}) as response:
        print(await response.json())

{'message': 'Hello John Smith!', 'parameters': {'extra-param': 'param-value'}}
